In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from pandas import DatetimeIndex
from sklearn.model_selection import cross_val_score

def mergeData(df):
    features =pd.read_csv('features.csv')
    storesdata =pd.read_csv('stores.csv')
    df = pd.merge(df, features, on=['Store','Date','IsHoliday'],how='inner')
    df = pd.merge(df, storesdata, on=['Store'], how='inner')
    return df


In [2]:
merged_df = mergeData(pd.read_csv('train.csv'))

In [3]:
merged_df.fillna(value=0, inplace=True)

merged_df['Markdowns'] = merged_df['MarkDown1'] + merged_df['MarkDown2'] + merged_df['MarkDown3'] + merged_df['MarkDown4'] + merged_df['MarkDown5'] 
labelsToDrop = ['MarkDown1', 'MarkDown2', 'MarkDown3','MarkDown4','MarkDown5']
merged_df.drop(labels=labelsToDrop,axis=1, inplace=True)
merged_df.loc[merged_df['Weekly_Sales']>300000,"Date"].value_counts()


2010-11-26    14
2011-11-25    13
2010-12-24     5
2011-12-23     2
Name: Date, dtype: int64

In [4]:
df = merged_df
print(df.head())

   Store  Dept        Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0      1     1  2010-02-05      24924.50      False        42.31       2.572   
1      1     2  2010-02-05      50605.27      False        42.31       2.572   
2      1     3  2010-02-05      13740.12      False        42.31       2.572   
3      1     4  2010-02-05      39954.04      False        42.31       2.572   
4      1     5  2010-02-05      32229.38      False        42.31       2.572   

          CPI  Unemployment Type    Size  Markdowns  
0  211.096358         8.106    A  151315        0.0  
1  211.096358         8.106    A  151315        0.0  
2  211.096358         8.106    A  151315        0.0  
3  211.096358         8.106    A  151315        0.0  
4  211.096358         8.106    A  151315        0.0  


In [5]:
df.Date = pd.to_datetime(df.Date)

df['Year'] = DatetimeIndex(df['Date']).year
df['Month']= DatetimeIndex(df['Date']).month
df['Day'] = DatetimeIndex(df['Date']).day
df = df.drop(columns=['Date'])

In [6]:
#creating the feature matrix 
feature_cols = ['Store', 'Temperature','Fuel_Price','CPI','Unemployment', 'Markdowns', 'Size', 'Dept', 'Weekly_Sales', 'Year', 'Month', 'Day']
X = df[feature_cols]
y = df['IsHoliday']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=2)

In [7]:
logreg = LogisticRegression()

In [8]:
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
y_predict_logreg = logreg.predict(X_test)
print(y_predict_logreg)

[False False False ... False False False]


In [10]:
score_logreg = accuracy_score(y_test, y_predict_logreg)
print(score_logreg)

0.9323244063856536


In [11]:
accuracy_list = cross_val_score(logreg, X, y, cv=5, scoring='accuracy')

In [12]:
accuracy_cv = accuracy_list.mean()

In [13]:
print(accuracy_cv)

0.9321963126221489
